In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal
import matplotlib.cm as cm
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D 
import sys
from decimal import Decimal
from IPython.display import clear_output
sys.path.append("..")
from kernels import vvkernels as vvk, sep_vvkernels as svvk, vvk_rbfkernel as vvk_rbf
from means import vvmeans as vvm
from likelihood import vvlikelihood as vvll
from mlikelihoods import MarginalLogLikelihood as exmll
from predstrategies import GPprediction
from utils import ObjFun, get_vertices, stopping_criteria
from scipy import stats
import numpy as np
import seaborn as sns
import scipy
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
vf = ObjFun()
f_target = vf.tgt_vec
print(f_target)
sample_size = 100
D = vf.D
N = vf.N

vf.low = -3.
vf.high = 3.

high_minus_low = vf.high- vf.low
def g_theta(sample_size, D):
    loc = high_minus_low  * np.random.random_sample((sample_size,2)) + vf.low#(np.random.uniform(low=vf.low, high=vf.high, size=(sample_size, D)))
    return Tensor(loc)
train_x = g_theta(sample_size, D)
#train_x = Tensor([[-1.5, 1.5], [1.5, -1.5]])
print(train_x)
noise_value = 0.0004 #noise_free = 0.
def vfield_(x):
    x = x.reshape(x.shape[0],D)
    out = torch.zeros(x.shape[0], N)
    out = vf(x[:,0], x[:,1]) + torch.randn(Tensor(vf(x[:,0], x[:,1])).size()) * math.sqrt(noise_value)
    return out #/torch.max(out)

train_y = vfield_(train_x)

tensor([0.3380, 0.3502], dtype=torch.float32)
tensor([[ 2.4683, -2.8006],
        [-2.1043, -0.8821],
        [-1.9342,  1.9788],
        [ 1.4148,  1.0389],
        [ 2.0117,  1.7284],
        [ 0.5297,  0.4172],
        [-0.6897, -1.4173],
        [ 0.5013, -2.9818],
        [-0.8649, -0.7607],
        [ 0.4400,  0.7817],
        [ 0.2724,  0.6959],
        [ 1.9166, -2.1338],
        [-0.3922,  2.3453],
        [ 2.4806, -0.4953],
        [-2.7516, -0.3190],
        [-2.4444, -0.9558],
        [-0.3199,  2.5898],
        [-0.1106, -2.2790],
        [-1.1771, -2.5949],
        [ 0.2845, -1.3441],
        [ 2.7202, -2.1631],
        [ 0.6443,  2.2437],
        [-1.4896, -1.4143],
        [ 2.9410, -1.4842],
        [-1.8677, -2.5650],
        [-0.3051, -2.8640],
        [ 2.8369, -2.4092],
        [ 0.2516,  0.4043],
        [-0.2993,  1.3328],
        [-1.0095,  0.2817],
        [-2.3846, -1.9468],
        [-2.5929, -1.3840],
        [ 0.3319,  0.1781],
        [ 1.3162, -1.0153],
  

In [4]:
x_train = train_x #loc #torch.linspace(0, 1, 10)
y_train = train_y #v  #torch.stack([torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,], -1)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood,num_base_kernels):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        a = torch.ones(2,2)
        chol_q = torch.tril(a)
        self.mean_module = vvm.TensorProductSubMean(gpytorch.means.LinearMean(2), num_tasks = 2)  #vvm.TensorProductSubMean(gpytorch.means.LinearMean(2), num_tasks = 2)#vvm.TensorProductSubMean(gpytorch.means.ConstantMean(), num_tasks = 2)  # 
        base_kernels = []
        for i in range(num_base_kernels):
            base_kernels.append(( gpytorch.kernels.RBFKernel() )) #gpytorch.kernels.PolynomialKernel(4)  ##gpytorch.kernels.MaternKernel()# (vvk_rbf.vvkRBFKernel())
#         base_kernels2 = []
#         for i in range(num_base_kernels):
#             base_kernels2.append(gpytorch.kernels.PolynomialKernel(5))  
            
        self.covar_module = svvk.SepTensorProductKernel(base_kernels,num_tasks = 2)
        #self.covar_module = gpytorch.kernels.LCMKernel(base_kernels,num_tasks = 2)

#\         self.covar_module = gpytorch.kernels.MultitaskKernel(
#             gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
#         )
       # self.covar_module = vvk.TensorProductKernel(vvk_rbf.vvkRBFKernel(), a[0,0], a[1,0], a[1,1], num_tasks = 2, rank =1,  task_covar_prior=None)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
    




In [5]:
# # ###hyperparameters optimization###
def hyper_opti(g_theta1, agg_data, training_iter,num_base_kernels,noise_value, current_model = None, current_likelihood = None):
    noises = torch.ones(agg_data.shape[0]) * (noise_value) #  torch.zeros(agg_data.shape[0]) # 
    noises = noises.reshape(g_theta1.shape[0], 2)
    
#     if (current_model is not None):
#         likelihood = current_likelihood #vvll.FixedNoiseMultitaskGaussianLikelihood(2, noises) #vvll.FixedNoiseMultitaskGaussianLikelihood(2, noises)  #

#         model = current_model#.get_fantasy_model(g_theta1, agg_data) #MultitaskGPModel(g_theta1, agg_data, likelihood,num_base_kernels)
#         model.set_train_data(g_theta1, agg_data,  strict=False)
#     else:
#         likelihood = vvll.FixedNoiseMultitaskGaussianLikelihood(noises) #vvll.TensorProductLikelihood(num_tasks = 2)#vvll.FixedNoiseMultitaskGaussianLikelihood(2, noises) #
#         model = MultitaskGPModel(g_theta1, agg_data, likelihood,num_base_kernels)
        
    likelihood =  vvll.FixedNoiseMultitaskGaussianLikelihood(noises) #vvll.TensorProductLikelihood(num_tasks = 2) #
    model = MultitaskGPModel(g_theta1, agg_data, likelihood,num_base_kernels)
#     model.double()
#     likelihood.double()


    model.train()
    
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(),  lr=0.1) #, weight_decay=0.001)  # Includes GaussianLikelihood parameters
    mll = exmll(likelihood, model)
   # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    #mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for i in range(training_iter):
        optimizer.zero_grad()
#         loss, inv_quad = likelihood.get_mll(agg_data,g_theta1, model, likelihood, noise_value)
#         loss = -loss
      #  output = model(g_theta1)
 #   Calc loss and backprop gradients
        #loss = -mll(output, agg_data)
        loss, chi_square = mll(agg_data,g_theta1, model, likelihood, noise_value)
        loss = -1. * loss
#         print('df is %.3f' %agg_data.shape[0] +'and chi_square %.3f' %chi_square) 
        print('loss is %.3f' %loss)
        df = agg_data.shape[0]
        chi_square = chi_square.clone().detach()
        
        p_val = 1. - stats.chi2.cdf(chi_square, df)
        loss.backward()
        optimizer.step()
        #scheduler.step(loss)
#         if (p_val > 0.99999):
#             return model, likelihood
        

    return model, likelihood

In [6]:
iter_hp = 200

num_base_kernels = 6





f_target = f_target.reshape(2,1) 







g_theta1 = x_train
agg_data = y_train.flatten()





    
    

cur_model = None
cur_likelihood = None



model, likelihood = hyper_opti(g_theta1,agg_data,iter_hp,num_base_kernels,noise_value, current_model = cur_model, current_likelihood = cur_likelihood)





loss is -0.148
loss is -0.377
loss is -0.602
loss is -0.818
loss is -1.017
loss is -1.192
loss is -1.332
loss is -1.428
loss is -1.471
loss is -1.463
loss is -1.424
loss is -1.384
loss is -1.374
loss is -1.399
loss is -1.447
loss is -1.501
loss is -1.545
loss is -1.571
loss is -1.579
loss is -1.576
loss is -1.571
loss is -1.573
loss is -1.584
loss is -1.606
loss is -1.636
loss is -1.670
loss is -1.700
loss is -1.722
loss is -1.735
loss is -1.743
loss is -1.754
loss is -1.771
loss is -1.795
loss is -1.818
loss is -1.837
loss is -1.849
loss is -1.854
loss is -1.860
loss is -1.872
loss is -1.891
loss is -1.911
loss is -1.926
loss is -1.936
loss is -1.944
loss is -1.954
loss is -1.968
loss is -1.984
loss is -1.990
loss is -1.995
loss is -2.006
loss is -2.019
loss is -2.027
loss is -2.028
loss is -2.036
loss is -2.047
loss is -2.049
loss is -2.052
loss is -2.058
loss is -2.064
loss is -2.064
loss is -2.069
loss is -2.072
loss is -2.074
loss is -2.077
loss is -2.081
loss is -2.081
loss is -2

In [7]:
from functools import partial
model.eval()
likelihood.eval()
x0 = Tensor(np.array([0.8731, 0.5664])) # 1./3. * Tensor(high_minus_low  * np.random.random_sample((1,2)) + vf.low) #
x0 = x0.reshape(1,2) #
fun = partial(likelihood.get_ell, agg_data,f_target, x0, g_theta1, model, likelihood, noise_value)



def tad_obj(x2, g_theta2):
    
    g_theta2_ = torch.cat([x2, g_theta2], 0)

    loss2, pf1, Qf1, Qf12, data_fit = fun(g_theta2_)
    return -loss2.detach()
err = 1e-5
tad_vec = torch.empty((1,1))
dist_vec = torch.empty((1,1))
g_theta2 = 1./3. *  Tensor(6.  * np.random.random_sample((50,2)) - 3.)
for i in range(60):
    
    x_plot = x0 + err
#     print(x_plot)
    dist = torch.norm(x0-x_plot).reshape(1,1)
    dist_vec = torch.cat([dist_vec, dist], 0)
    tad_val = tad_obj(x_plot, g_theta2).reshape(1,1)
    tad_vec = torch.cat([tad_vec, tad_val], 0)
    err = err*1.2


pooiiiib
tensor([[0.8731, 0.5664]])
tensor([[ 0.8731,  0.5664],
        [-0.3929, -0.3277],
        [ 0.1632,  0.3333],
        [-0.1885, -0.4165],
        [-0.7430, -0.0559],
        [-0.9464,  0.3909],
        [ 0.6872, -0.3770],
        [ 0.4608,  0.5156],
        [ 0.5210,  0.7621],
        [ 0.6418, -0.6243],
        [-0.4407,  0.1266],
        [-0.2890, -0.0111],
        [ 0.0717,  0.8106],
        [ 0.2333, -0.8968],
        [ 0.2710, -0.1219],
        [-0.0174, -0.4012],
        [-0.8329, -0.0578],
        [ 0.2581,  0.8180],
        [ 0.6606, -0.8927],
        [-0.3964, -0.7127],
        [-0.9927, -0.1078],
        [-0.6344,  0.3009],
        [-0.7357, -0.5184],
        [-0.9433,  0.5205],
        [ 0.2129,  0.9496],
        [-0.1910,  0.5101],
        [-0.9870,  0.5820],
        [ 0.1765, -0.5181],
        [ 0.7720, -0.3344],
        [ 0.7360, -0.5933],
        [ 0.0787,  0.7088],
        [-0.7667,  0.7035],
        [ 0.4420,  0.9784],
        [ 0.5938, -0.4767],
        [ 0.

ValueError: too many values to unpack (expected 5)

In [ ]:
plt.rcParams["pdf.use14corefonts"] = True

f, ax = plt.subplots(1, 1, figsize=(12, 12))
#ax.plot(np.array(range(2,iter+2)), torch.log(entropy_vec_plot), '+-')

ax.plot(np.array(dist_vec[1:]), (tad_vec[1:]),'-o',color = 'blue', markersize=12)
#ax.set_xlim(-15.1, torch.max(data_fit_vec_plot)+0.1)
#ax.set_yscale('log')
ax.set_xlabel('distance')
ax.set_ylabel('TAD')
#ax.plot(np.array(range(2,iter+2)), data_fit_vec_plot, 'rx-')
#plt.savefig('figures/TADvsDIST.eps',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:


def tad_obj2(g_theta2_):
    

    loss2, pf1, Qf1, Qf12, data_fit = fun(g_theta2_)
    return -loss2.detach()
err = 1e-5
tad_vec = torch.empty((1,1))
dist_vec = torch.empty((1,1))
#g_theta2 = 1./3. *  Tensor(6.  * np.random.random_sample((3,2)) - 3.)
for i in range(60):
    
    x_plot = (g_theta2[0] + err).reshape(1,2)
    g_theta2_ = torch.cat([x_plot, g_theta2], 0)
    print(x_plot)
    dist = torch.norm(g_theta2[0].reshape(1,2)-x_plot).reshape(1,1)
    dist_vec = torch.cat([dist_vec, dist], 0)
    tad_val = tad_obj2(g_theta2_).reshape(1,1)
    tad_vec = torch.cat([tad_vec, tad_val], 0)
    err = err*1.2

In [ ]:
plt.rcParams["pdf.use14corefonts"] = True

f, ax = plt.subplots(1, 1, figsize=(12, 12))
#ax.plot(np.array(range(2,iter+2)), torch.log(entropy_vec_plot), '+-')

ax.plot(np.array(dist_vec[1:]), (tad_vec[1:]),'-o',color = 'blue', markersize=12)
#ax.set_xlim(-15.1, torch.max(data_fit_vec_plot)+0.1)
#ax.set_yscale('log')
ax.set_xlabel('distance')
ax.set_ylabel('TAD')
#ax.plot(np.array(range(2,iter+2)), data_fit_vec_plot, 'rx-')
#plt.savefig('figures/TADvsDIST.eps',dpi=300, bbox_inches='tight')
plt.show()

In [12]:
x1 = Tensor([[2.5000, 1.5498]])
x2 = Tensor([[1.5323, 1.7661]])
res = torch.norm(x1 - x2, p=2, dim=-1)

In [13]:
print(res)

tensor([0.9916])


In [10]:
res = torch.norm(x1 - x2)

In [11]:
print(res)

tensor(1.0836)
